In [14]:
import os
import numpy as np

In [15]:

def count_adjacent_true(arr):
    assert len(arr.shape) == 1
    assert arr.dtype == np.bool
    if arr.size == 0:
        return np.empty(0, dtype=int), np.empty(0, dtype=int)
    sw = np.insert(arr[1:] ^ arr[:-1], [0, arr.shape[0]-1], values=True)
    swi = np.arange(sw.shape[0])[sw]
    offset = 0 if arr[0] else 1
    lengths = swi[offset+1::2] - swi[offset:-1:2]
    if lengths.shape[0]==0:
        return (np.array([0]),np.array([0]))
    else:
        return swi[offset:-1:2], lengths
    

a=np.zeros(100)
a[50:]=1
def compute_metrics_label(label,search='01'):
    label=(label==1)*1
    l=len(label)
    adj=count_adjacent_true(label==1)
    
    string=np.array2string(label,precision=0,separator='')[1:-1].replace('.','').replace(' ','')
    pattern=string.count(search)
    
#     print(adj)
    return np.array([min(adj[1])/l,max(adj[1])/l,np.mean(adj[1])/l,label.sum()/l,label[3::4].sum()/l,label.sum(),pattern])
res=compute_metrics_label(a)
print(res)

[ 0.5   0.5   0.5   0.5   0.13 50.    1.  ]


In [3]:
path_tags= [
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Blues.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Country.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Electronic.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Folk.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Jazz.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Latin.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Metal.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_New-Age.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Pop.id', # 8
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Punk.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Rap.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Reggae.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_RnB.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Rock.id', # 13
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_World.id',
    '/home/ftamagnan/dataset/id_lists/tagtraum/tagtraum_Unknown.id'
]
filepath_dataset = '/home/ftamagnan/dataset/lpd_5/lpd_5_cleansed/'

In [4]:
def iterate_metrics_fusion():

    metrics=np.zeros((1,8))
    for tag_i, tag in enumerate(path_tags):


        print('>>' + tag[32:-3])
        with open(tag, 'r') as f:
            # ITERATE OVER THE FOLDER LISTS
            for i, file in enumerate(f):
                file = file.rstrip()
                middle = '/'.join(file[2:5]) + '/'        
                p = filepath_dataset + middle + file
                for npz in os.listdir(p):
                    if 'label.npz' in npz:
                        data=dict(np.load(p+'/'+npz))
                        label_s=data['label']
                    if 'label_clustering.npz' in npz:
                        data=dict(np.load(p+'/'+npz))
                        label_c=data['label']
                label_c=label_c==1*1
                label=np.logical_and(label_c,label_s)*1
                m=compute_metrics_label(label)
                m_=np.concatenate([m,np.array([tag_i])])

                m_=m_.reshape((1,8))
                metrics=np.concatenate((metrics,m_))
                
                        
    return metrics

In [4]:
def iterate_metrics(labeltype,search='01'):

    metrics=np.zeros((1,8))
    for tag_i, tag in enumerate(path_tags):


        print('>>' + tag[32:-3])
        with open(tag, 'r') as f:
            # ITERATE OVER THE FOLDER LISTS
            for i, file in enumerate(f):
                file = file.rstrip()
                middle = '/'.join(file[2:5]) + '/'        
                p = filepath_dataset + middle + file
                for npz in os.listdir(p):
                    if labeltype in npz:
                        data=dict(np.load(p+'/'+npz))
                        label=data['label']
                        m=compute_metrics_label(label,search=search)
                        m_=np.concatenate([m,np.array([tag_i])])
                        m_=m_.reshape((1,8))
                        
                        metrics=np.concatenate((metrics,m_))
    return metrics

In [22]:
# metrics_fusion=iterate_metrics_fusion()
metrics_sk=iterate_metrics('label_clustering.npz')
# metrics_clustering=iterate_metrics('label_clustering.npz',search='-11')



>>/tagtraum/tagtraum_Blues
>>/tagtraum/tagtraum_Country
>>/tagtraum/tagtraum_Electronic
>>/tagtraum/tagtraum_Folk
>>/tagtraum/tagtraum_Jazz
>>/tagtraum/tagtraum_Latin
>>/tagtraum/tagtraum_Metal
>>/tagtraum/tagtraum_New-Age
>>/tagtraum/tagtraum_Pop
>>/tagtraum/tagtraum_Punk
>>/tagtraum/tagtraum_Rap
>>/tagtraum/tagtraum_Reggae
>>/tagtraum/tagtraum_RnB
>>/tagtraum/tagtraum_Rock
>>/tagtraum/tagtraum_World
>>/tagtraum/tagtraum_Unknown


In [23]:
np.savez('./metrics_clustering.npz',metrics_sk=metrics_sk)

In [8]:
import matplotlib.pyplot as plt

data = {'apples': 10, 'oranges': 15, 'lemons': 5, 'limes': 20}
names = list(data.keys())
values = list(data.values())

fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharey=True)
axs[0].bar(names, values)
axs[1].scatter(names, values)
axs[2].plot(names, values)
fig.suptitle('Categorical Plotting')


ValueError: could not convert string to float: 'limes'